<a href="https://colab.research.google.com/github/Goodman2442/test11/blob/main/nice_cc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip  install openai >/dev/null

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import json
import openai
import os

In [ ]:
#@title Задаем константы
gpt_4_turbo = 'gpt-4o'
gpt_35_turbo = 'gpt-3.5-turbo-1106'
MODEL = gpt_4_turbo

temp = 0.1
verbose = 1

In [ ]:
#@title  Используем секретный ключ в колабе
from google.colab import userdata
key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = key

In [ ]:
#@title Создание чеклиста таблицы
table_data = [
  {
    "Группа": "Форма обращения к клиенту",
    "Критерий для оценки": "Используется уважительная форма обращения на \"Вы\".",
    "Пример из разговора": "",
    "Оценка": ""
  },
  {
    "Группа": "Выявление запретных фраз",
    "Критерий для оценки": "Исключены слова, выражающие неуверенность: \"Кажется\", \"Возможно\", и т.д.",
    "Пример из разговора": "",
    "Оценка": ""
  },
  {
    "Группа": "Выявление запретных фраз",
    "Критерий для оценки": "Исключены фразы, вызывающие раздражение у клиента: \"Как правило\", \"Несмотря на перечисленное\", и т.д.",
    "Пример из разговора": "",
    "Оценка": ""
  },
  {
    "Группа": "Выявление запретных фраз",
    "Критерий для оценки": "Исключены слова-паразиты: \"Ага\", \"Ну\", \"О\", \"Ох\", \"Эх\", и т.д.",
    "Пример из разговора": "",
    "Оценка": ""
  },
  {
    "Группа": "Выявление запретных фраз",
    "Критерий для оценки": "Исключены выражения: \"Как бы\", \"Если бы\", \"Как Вам сказать\", \"Вот это вот\" и аналогичные.",
    "Пример из разговора": "",
    "Оценка": ""
  },
  {
    "Группа": "Ответы на вопросы клиента",
    "Критерий для оценки": "Если оператор не знает ответа, используются правильные фразы: \"Это не в моей компетенции, но я могу уточнить информацию у мастера для вас.\", \"На этот вопрос я ответить не могу, но сейчас я уточню и вернусь к вам с ответом.\"",
    "Пример из разговора": "",
    "Оценка": ""
  },
 {
    "Группа": "Ответы на вопросы клиента",
    "Критерий для оценки": "Клиент удовлетворен ответом менеджера и благодарен ему за предоставленную информацию",
    "Пример из разговора": "",
    "Оценка": ""
  },
  {
    "Группа": "Уменьшительно-ласкательная форма",
    "Критерий для оценки": "Уменьшительно-ласкательная форма запрещена к применению в разговоре.",
    "Пример из разговора": "",
    "Оценка": ""
  },
  {
    "Группа": "Использование частицы \"Не\"",
    "Критерий для оценки": "Частьцы \"Не\" в вопросах к клиенту избегаются:",
    "Пример из разговора": "",
    "Оценка": ""
  },
  {
    "Группа": "Использование частицы \"Не\"",
    "Критерий для оценки": "Вместо \"Не хотели ли бы вы...\" используется \"Хотите ли вы...\", Вместо \"Не могли бы вы...\" используется \"Могли бы вы...\", Вместо \"Вас не интересует...\" используется \"Интересует ли вас...\"",
    "Пример из разговора": "",
    "Оценка": ""
  },
  {
    "Группа": "Споры с клиентом",
    "Критерий для оценки": "Избегаются фразы: \"Это не так\", \"Это невозможно\"",
    "Пример из разговора": "",
    "Оценка": ""
  },
  {
    "Группа": "Споры с клиентом",
    "Критерий для оценки": "Правильная реакция на возражения о цене: \"Я с вами полностью согласен, но...\"",
    "Пример из разговора": "",
    "Оценка": ""
  },
  {
    "Группа": "Споры с клиентом",
    "Критерий для оценки": "Объяснение ценности продукта клиенту и оправдание его цены.",
    "Пример из разговора": "",
    "Оценка": ""
  },
  {
    "Группа": "Категория обращения",
    "Критерий для оценки": "Ремонт/сервис",
    "Пример из разговора": "",
    "Оценка": ""
  },
  {
    "Группа": "Категория обращения",
    "Критерий для оценки": "Продажа товаров Apple",
    "Пример из разговора": "",
    "Оценка": ""
  },
  {
    "Группа": "Категория обращения",
    "Критерий для оценки": "Продажа других товаров",
    "Пример из разговора": "",
    "Оценка": ""
  },
  {
    "Группа": "Категория обращения",
    "Критерий для оценки": "Предоставление общей информации",
    "Пример из разговора": "",
    "Оценка": ""
  },
  {
    "Группа": "Оценка работы менеджера",
    "Критерий для оценки": "Предоставление всех преимуществ покупки дорогих товаров.",
    "Пример из разговора": "",
    "Оценка": ""
  },
  {
    "Группа": "Ответы на вопросы клиента",
    "Критерий для оценки": "Оценка полноты и точности ответов на все вопросы клиента.",
    "Пример из разговора": "",
    "Оценка": ""
  },
  {
    "Группа": "Работа с возражениями клиента",
    "Критерий для оценки": "Отработка возражений клиента менеджером с предоставлением аргументов",
    "Пример из разговора": "",
    "Оценка": ""
  }
]


In [ ]:
#@title System & User
system ='''Ты специалист контроля качества контакт центра. Ты оцениваешь работу менеджеров заполняя чеклисту. В колонку "Оценка" поставь 1, если пункт выполнен успешно, 0 провален.
            В поле "Примеры из текста" добавляешь подтверждения поставленной оценки примером из разговора. Делаешь краткое резюме:предоставляешь общую характеристику работы менеджера и что можно улучшить.'''
instructions = '''Проанализируй разговор и заполни чеклист в формате CSV разделитель | и формат чеклиста. Добавь один общий абзац для резюме звонка '''

In [ ]:
#@title Функция для анализа текста
def analyze_text(text, system, instructions, temp=0, verbose=1, model=MODEL):

    if verbose: print('==================')

    messages = [

        {"role": "system", "content": f''' {system}
          Чек-лист для проверки коммуникации менеджера с клиентом\n: {table_data}
           '''},

        {"role": "user", "content": f'''{instructions}

         Разговор клиента с менеджером: {text}'''}

    ]
    print()
    completion = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp
    )

    if verbose: print('\n==================')
    if verbose:
      print(f'{completion.usage.total_tokens} total tokens used (question-answer).')

    return completion.choices[0].message.content

In [ ]:
#@title Чтение из файла, анализ текста и вывод результата

import urllib.request
import json
import pandas as pd
from io import StringIO  # Import StringIO

# Файл с транскрибаций от @o_shadr
SHEET_URL = "https://docs.google.com/spreadsheets/d/1Z83k0k8GRa6p82CzZo3kDKK3WclcvTvR/export?format=csv"

# Read the CSV data from the Google Sheet
with urllib.request.urlopen(SHEET_URL) as response:
    data = response.read().decode('utf-8')

# Convert CSV to DataFrame
df = pd.read_csv(StringIO(data))

# Define start and end lines of the dataframe
start_line = 10
end_line = 12

# Iterate through the specified range
for i in range(start_line, end_line + 1):
    text = df.iloc[i-1, 10] #только по выбранному диапазону
    #text = df.iloc[:, 10] #чтобы пройти по всей колонке с текстами звонков
    analysis = analyze_text(text, system, instructions)
    print(analysis)
    #analysis_json = json.loads(analysis)
    # Print JSON as a table (using pandas for easy formatting)
    #print(pd.DataFrame(analysis_json).to_string())





2288 total tokens used (question-answer).
Форма обращения к клиенту|1
Выявление запретных фраз|1
Ответы на вопросы клиента|1
Уменьшительно-ласкательная форма|1
Использование частицы "Не"|1
Споры с клиентом|1
Категория обращения|1
Оценка работы менеджера|1
Работа с возражениями клиента|0

Общее впечатление: Менеджер вежливо общался с клиентом, использовал уважительную форму обращения и предоставил информацию о заказе. Однако, не были отработаны возможные возражения клиента, что может быть улучшено.


2423 total tokens used (question-answer).
Форма обращения к клиенту|1
Выявление запретных фраз|1
Ответы на вопросы клиента|0
Уменьшительно-ласкательная форма|1
Использование частицы "Не"|1
Споры с клиентом|1
Категория обращения|1
Оценка работы менеджера|0
Работа с возражениями клиента|0

Общее резюме: Менеджер использовал уважительную форму обращения, избегал запретных фраз и споров с клиентом. Однако, не смог предоставить полные и точные ответы на вопросы клиента, а также не отработал во